In [1]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
os.chdir(current_dir)

In [5]:
ls

bernardo_35.txt      carl_36.txt    gold_psy.pkl    mitchell_34.txt    wayne_40.txt
bow_baselines.ipynb  dahmer_23.txt  kemper_26.txt   pos_dataframe.pkl
bundy_34.txt         gacy_27.txt    marsh_34.5.txt  ridgeway_19.txt


In [6]:
import pandas as pd

# Load the pickled DataFrame
copy = pd.read_pickle("pos_dataframe.pkl")


In [7]:
copy

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

  trinary     group                                                ADJ  \
0    high       doc  first criminal other little more dead opposite...   
1    high       doc  statutory minor sexual female old good female ...   
2    high  handbook  true wonderful dedicated young young local loc...   
3    high       cbc  peel regional public peel regional big crazy c...   
4  medium  handbook  lucky crazy slow steady ordinary last last int...   
5  medium  handbook  low last simple homosexual young correct sound...   
6  medium  handbook  more impossible nice ordinary most sick other ...   
7    high  articles  numerous numerous horrible terrible personal f...   
8    high  articles  nice only right almighty holy true holy other ...   
9     low  handbook  fourth fourth fourth fifth sixth angry alone a...   

                                                 ADP  \
0  of of with on down up with of off on for besid...   
1                       on for on on around of to at   
2  up in with of as of outside in to in of out of...   
3  about with of on for by around on to on with f...   
4  up on like out of to about to of of off for of...   
5  from in of against in on for to for for for of...   
6  for than in up up in out of in as of on in out...   
7  at of of in into with into into like at with o...   
8  in of with with about through through of by of...   
9  to in at for in of at for on during during out...   

                                                 ADV  ...  \
0  very together away ever so just so right sure ...  ...   
1                        then someday very at around  ...   
2  certainly again more generally more more more ...  ...   
3  aka forward maybe here just in back there here...  ...   
4  billy up very just just very so thoroughly jus...  ...   
5  strictly constantly constantly right always aw...  ...   
6  almost far that even increasingly back further...  ...   
7  just everywhere usually just there probably us...  ...   
8  as absolutely so so now once again still very ...  ...   
9  really just just probably probably too so just...  ...   

                                                 NUM  \
0     five twenty four four twelve forty five twenty   
1                                           four one   
2  two one five twelve thirteen one twenty thirty...   
3                          seventeen one one one two   
4  f

In [9]:
# Create a DataFrame

df = copy

# Extract rows for the test set
test_df = df[df['name'].isin(['ridgeway', 'wayne', 'carl'])]

# Extract the rest of the data for the training set
train_df = df[~df['name'].isin(['ridgeway', 'wayne', 'carl'])]


In [48]:
train_df

name  score                                               text  \
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   

                                          clean_text            binary  \
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   

  trinary     group                                                ADJ  \
2    high  handbook  true wonderful dedicated young young local loc...   
3    high       cbc  peel regional public peel regional big crazy c...   
4  medium  handbook  lucky crazy slow steady ordinary last last int...   
5  medium  handbook  low last simple homosexual young correct sound...   
6  medium  handbook  more impossible nice ordinary most sick other ...   
7    high  articles  numerous numerous horrible terrible personal f...   
8    high  articles  nice only right almighty holy true holy other ...   

                                                 ADP  \
2  up in with of as of outside in to in of out of...   
3  about with of on for by around on to on with f...   
4  up on like out of to about to of of off for of...   
5  from in of against in on for to for for for of...   
6  for than in up up in out of in as of on in out...   
7  at of of in into with into into like at with o...   
8  in of with with about through through of by of...   

                                                 ADV  ...  \
2  certainly again more generally more more more ...  ...   
3  aka forward maybe here just in back there here...  ...   
4  billy up very just just very so thoroughly jus...  ...   
5  strictly constantly constantly right always aw...  ...   
6  almost far that even increasingly back further...  ...   
7  just everywhere usually just there probably us...  ...   
8  as absolutely so so now once again still very ...  ...   

                                                 NUM  \
2  two one five twelve thirteen one twenty thirty...   
3                          seventeen one one one two   
4  fourteen fifteen fifteen three eighteen five n...   
5  twelve nineteen seventy eight three million fo...   
6  twenty thirty five one one one seventy two one...   
7                                  two seventeen one   
8  ten eighteen fifty three eighteen eighteen eig...   

                                                PART  \
2  to nt to to to to nt nt nt nt to not not nt nt...   
3  nt to not nt nt na to to nt nt nt nt not to no...   
4  nt to to not not to not nt nt nt nt to to to n...   
5  not to to na nt na nt to not to nt to to nt to...   
6  not not not nt nt nt to to not nt to to not to...   
7  nt to to nt to not nt nt nt to nt not to nt nt...   
8  to nt to to to not to to not to nt not to not ...   

                                                PRON  \
2  that i i i what i i i you we our whatever they...   
3  i it what that you i i there any i i they they...   
4  you you there it nothing nothing i my it me it...   
5  that that i it i i i i this i its i my it they...   
6  i i i someone who it its its it it they he she...   
7  i someone i me them i it its what my i them i ...   
8  that 

In [8]:
train_df.columns

NameError: ignored

In [88]:
from itertools import combinations

all_groupings = ['ADV', 'AUX', 'NUM', 'VERB']




19


In [89]:


# Outer loop for different feature column combinations
for feature_columns in all_groupings:
    print("Feature Columns:", feature_columns)

    # Split the data into features and target variable for training
    X_train = train_df[feature_columns]
    y_train = train_df['score']

    # Initialize variables to keep track of the best model and its performance
    best_model = None
    best_score = float('-inf')

    # Example of iterating over the models and creating a pipeline for each
    for model in models:
        pipeline = create_pipeline(model, feature_columns)

        # Perform 7-fold cross-validation and calculate negative mean squared error
        scores = cross_val_score(pipeline, X_train, y_train, cv=6, scoring='neg_mean_squared_error')

        # Calculate the average negative mean squared error
        avg_score = scores.mean()

        # Check if this model is better than the current best model
        if avg_score > best_score:
            best_score = avg_score
            best_model = model

    # Fit the best model on the entire training dataset
    best_pipeline = create_pipeline(best_model, feature_columns)
    best_pipeline.fit(X_train, y_train)

    # Now, you can evaluate the best model on a separate test set
    X_test = test_df[feature_columns]
    y_test = test_df['score']
    names_test = test_df['name']  # Add this line to get subject names

    # Make predictions on the test set using the best model
    y_pred = best_pipeline.predict(X_test)

    # Print the name of the subject, predicted score, and actual score
    for name, predicted, actual in zip(names_test, y_pred, y_test):
        print("Name:", name)
        print("Predicted Score:", predicted)
        print("Actual Score:", actual)
        print()  # Empty line for separation

    # Calculate the mean squared error on the test set
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the R-squared score on the test set
    r2 = r2_score(y_test, y_pred)

    # Print the best model and its performance including R-squared
    print("Best Model:", best_model)
    print("Best Negative Mean Squared Error:", best_score)
    print("Mean Squared Error on Test Set:", mse)
    print("R-squared on Test Set:", r2)
    print()



Feature Columns: ['ADV']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.323e-02, tolerance: 8.388e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.623e-02, tolerance: 8.388e-03
  model = cd_fast.enet_coordinate_descent(


Name: wayne
Predicted Score: 31.215
Actual Score: 40.0

Name: carl
Predicted Score: 31.38
Actual Score: 36.0

Name: ridgeway
Predicted Score: 29.55
Actual Score: 19.0

Best Model: RandomForestRegressor()
Best Negative Mean Squared Error: -35.51232708333334
Mean Squared Error on Test Set: 69.94104166666666
R-squared on Test Set: 0.15620727211796248

Feature Columns: ['AUX']
Name: wayne
Predicted Score: 37.15356032228683
Actual Score: 40.0

Name: carl
Predicted Score: 37.46825574293553
Actual Score: 36.0

Name: ridgeway
Predicted Score: 20.474703027905647
Actual Score: 19.0

Best Model: LassoCV()
Best Negative Mean Squared Error: -5.381071690088952
Mean Squared Error on Test Set: 4.144247595345706
R-squared on Test Set: 0.9500023748550787

Feature Columns: ['NUM']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Name: wayne
Predicted Score: 26.0
Actual Score: 40.0

Name: carl
Predicted Score: 23.0
Actual Score: 36.0

Name: ridgeway
Predicted Score: 34.0
Actual Score: 19.0

Best Model: DecisionTreeRegressor()
Best Negative Mean Squared Error: -24.5
Mean Squared Error on Test Set: 196.66666666666666
R-squared on Test Set: -1.3726541554959786

Feature Columns: ['VERB']
Name: wayne
Predicted Score: 34.56565220135147
Actual Score: 40.0

Name: carl
Predicted Score: 34.59135597888644
Actual Score: 36.0

Name: ridgeway
Predicted Score: 28.961768832268362
Actual Score: 19.0

Best Model: ElasticNet()
Best Negative Mean Squared Error: -13.346534069689868
Mean Squared Error on Test Set: 43.58441741414949
R-squared on Test Set: 0.47418263173278097

Feature Columns: ['ADV', 'AUX']
Name: wayne
Predicted Score: 36.826706774963554
Actual Score: 40.0

Name: carl
Predicted Score: 37.12594290621183
Actual Score: 36.0

Name: ridgeway
Predicted Score: 20.967191818804917
Actual Score: 19.0

Best Model: LassoCV()
Bes

...

Name: wayne
Predicted Score: 36.170170876250076
Actual Score: 40.0

Name: carl
Predicted Score: 36.51665321573992
Actual Score: 36.0

Name: ridgeway
Predicted Score: 21.149062092778163
Actual Score: 19.0

Best Model: ElasticNet()
Best Negative Mean Squared Error: -3.2947968480541356
Mean Squared Error on Test Set: 6.517663180357857
R-squared on Test Set: 0.9213686747678007

Feature Columns: ['ADV']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.323e-02, tolerance: 8.388e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.623e-02, tolerance: 8.388e-03
  model = cd_fast.enet_coordinate_descent(


Name: wayne
Predicted Score: 31.1
Actual Score: 40.0

Name: carl
Predicted Score: 31.1
Actual Score: 36.0

Name: ridgeway
Predicted Score: 29.1
Actual Score: 19.0

Best Model: KNeighborsRegressor()
Best Negative Mean Squared Error: -36.87666666666669
Mean Squared Error on Test Set: 68.41000000000001
R-squared on Test Set: 0.17467828418230558

Feature Columns: ['AUX']
Name: wayne
Predicted Score: 37.15356032228683
Actual Score: 40.0

Name: carl
Predicted Score: 37.46825574293553
Actual Score: 36.0

Name: ridgeway
Predicted Score: 20.474703027905647
Actual Score: 19.0

Best Model: LassoCV()
Best Negative Mean Squared Error: -5.381071690088952
Mean Squared Error on Test Set: 4.144247595345706
R-squared on Test Set: 0.9500023748550787

Feature Columns: ['NUM']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Name: wayne
Predicted Score: 31.788388130199912
Actual Score: 40.0

Name: carl
Predicted Score: 34.03064073582829
Actual Score: 36.0

Name: ridgeway
Predicted Score: 30.782542841188324
Actual Score: 19.0

Best Model: Ridge()
Best Negative Mean Squared Error: -24.838753294472877
Mean Squared Error on Test Set: 70.04575373868627
R-squared on Test Set: 0.1549439897477528

Feature Columns: ['VERB']
Name: wayne
Predicted Score: 34.56565220135147
Actual Score: 40.0

Name: carl
Predicted Score: 34.59135597888644
Actual Score: 36.0

Name: ridgeway
Predicted Score: 28.961768832268362
Actual Score: 19.0

Best Model: ElasticNet()
Best Negative Mean Squared Error: -13.346534069689868
Mean Squared Error on Test Set: 43.58441741414949
R-squared on Test Set: 0.47418263173278097

